# Assingment 4 -- By Aryan Rathore

## 1. open the files in the meddra and original2 folders got predicted_test and truth_text
## 2. word wise split both the pred_text and truth_Text
## 3. code for precision recall and F1 + why these are used
## 4. final predictions

## --- 1. open the files in the meddra and original2 folders got predicted_test and truth_text ---

In [37]:
import pandas as pd

# reading the predicted labels from original2 folder and making it a df

pred_text1 = pd.read_csv("/content/drive/MyDrive/cadec/original2/ARTHROTEC.1.txt")
pred_text2 = pd.read_csv("/content/drive/MyDrive/cadec/original2/ARTHROTEC.2.txt")
pred_text3 = pd.read_csv("/content/drive/MyDrive/cadec/original2/ARTHROTEC.3.txt")
pred_text4 = pd.read_csv("/content/drive/MyDrive/cadec/original2/ARTHROTEC.4.txt")
pred_text5 = pd.read_csv("/content/drive/MyDrive/cadec/original2/ARTHROTEC.5.txt")

pred_text1['text'] = pred_text1['text'].str.lower()
pred_text2['text'] = pred_text2['text'].str.lower()
pred_text3['text'] = pred_text3['text'].str.lower()
pred_text4['text'] = pred_text4['text'].str.lower()
pred_text5['text'] = pred_text5['text'].str.lower()

pred_text1 = pred_text1[pred_text1['identifier'] == "ADR"]
pred_text2 = pred_text2[pred_text2['identifier'] == "ADR"]
pred_text3 = pred_text3[pred_text3['identifier'] == "ADR"]
pred_text4 = pred_text4[pred_text4['identifier'] == "ADR"] # empty because nothing marked as ADR
pred_text5 = pred_text5[pred_text5['identifier'] == "ADR"] # empty because nothing marked as ADR


# reading the Ground truth from meddra folder and making it a df

def read_ann_to_dataframe(filepath):
    """
    Reads a .ann file and returns a pandas DataFrame.

    input = filepath (str) : output = DataFrame

    """
    data = []
    with open(filepath, "r") as file:
        texts = file.read().splitlines()
        texts = [line.split("\t") for line in texts if line and not line.startswith('#')]
        for entry in texts:
            my_dict = {}
            my_dict['file_name'] = filepath.split('/')[-1].lower() # Extract filename from path
            my_dict["identifier"] = "ADR"
            my_dict["text"] = entry[2].lower()
            data.append(my_dict)
    truth_text_df = pd.DataFrame(data, columns=["identifier", "text", "file_name"])
    return truth_text_df


truth_text1 = read_ann_to_dataframe("/content/drive/MyDrive/cadec/meddra/ARTHROTEC.1.ann")
truth_text2 = read_ann_to_dataframe("/content/drive/MyDrive/cadec/meddra/ARTHROTEC.2.ann")
truth_text3 = read_ann_to_dataframe("/content/drive/MyDrive/cadec/meddra/ARTHROTEC.3.ann")
truth_text4 = read_ann_to_dataframe("/content/drive/MyDrive/cadec/meddra/ARTHROTEC.4.ann")
truth_text5 = read_ann_to_dataframe("/content/drive/MyDrive/cadec/meddra/ARTHROTEC.5.ann")

In [38]:
truth_text3

,identifier,text,file_name
0,ADR,constipation,arthrotec.3.ann
1,ADR,joint pain,arthrotec.3.ann
2,ADR,pain,arthrotec.3.ann
3,ADR,extreme itching on legs and arms,arthrotec.3.ann
4,ADR,mobility has decreased,arthrotec.3.ann
5,ADR,constipation,arthrotec.3.ann
6,ADR,muscle pain,arthrotec.3.ann
7,ADR,joint pain,arthrotec.3.ann


In [39]:
pred_text3

,identifier,text,file_name
0,ADR,constipation,ARTHROTEC.3.txt
3,ADR,extreme itching,ARTHROTEC.3.txt
7,ADR,muscle/joint pain,ARTHROTEC.3.txt


## --- 2. word wise split both the pred_text and truth_Text ---

## for better calculations i am diluting the text columns in both pred_text and truth_text so that each word has a label

In [41]:
# Function to split text and create new rows
def split_text_to_rows(df):
    new_data = []
    for index, row in df.iterrows():
        identifier = row['identifier']
        text_words = row['text'].split()
        for word in text_words:
            new_data.append({'identifier': identifier, 'text': word.lower()}) # Added .lower() here
    return pd.DataFrame(new_data)

# Split text and remove duplicates for pred_text1

pred_text1_split = split_text_to_rows(pred_text1)
pred_text1_split_unique = pred_text1_split.drop_duplicates().reset_index(drop=True)

pred_text2_split = split_text_to_rows(pred_text2)
pred_text2_split_unique = pred_text2_split.drop_duplicates().reset_index(drop=True)

pred_text3_split = split_text_to_rows(pred_text3)
pred_text3_split_unique = pred_text3_split.drop_duplicates().reset_index(drop=True)

pred_text4_split = split_text_to_rows(pred_text4)
pred_text4_split_unique = pred_text4_split.drop_duplicates().reset_index(drop=True)


# Split text and remove duplicates for truth_text1

truth_text1_split = split_text_to_rows(truth_text1)
truth_text1_split_unique = truth_text1_split.drop_duplicates().reset_index(drop=True)

truth_text2_split = split_text_to_rows(truth_text2)
truth_text2_split_unique = truth_text2_split.drop_duplicates().reset_index(drop=True)

truth_text3_split = split_text_to_rows(truth_text3)
truth_text3_split_unique = truth_text3_split.drop_duplicates().reset_index(drop=True)

truth_text4_split = split_text_to_rows(truth_text4)
truth_text4_split_unique = truth_text4_split.drop_duplicates().reset_index(drop=True)



print("Processed pred_text3:")
display(pred_text3_split_unique)

print("\nProcessed truth_text3:")
display(truth_text3_split_unique)

Processed pred_text3:


,identifier,text
0,ADR,constipation
1,ADR,extreme
2,ADR,itching
3,ADR,muscle/joint
4,ADR,pain



Processed truth_text3:


,identifier,text
0,ADR,constipation
1,ADR,joint
2,ADR,pain
3,ADR,extreme
4,ADR,itching
5,ADR,on
6,ADR,legs
7,ADR,and
8,ADR,arms
9,ADR,mobility


# --- 3. code for precision recall and F1 + why these are used ---

In [42]:
import pandas as pd

def calculate_ner_performance(predicted_df, truth_df):
    """
    Calculates Precision, Recall, and F1-Score for NER performance at the word level.

    Args:
        predicted_df (pd.DataFrame): DataFrame with predicted labels and words (split and unique).
        truth_df (pd.DataFrame): DataFrame with ground truth labels and words (split and unique).

    Returns:
        dict: A dictionary containing Precision, Recall, and F1-Score.

    Justification for using Precision, Recall, and F1-Score:
    These metrics are commonly used in evaluating information extraction tasks like Named Entity Recognition (NER).
    - Precision measures the accuracy of the positive predictions (how many of the predicted entities are correct).
    - Recall measures the ability of the model to find all the positive instances (how many of the actual entities were predicted).
    - The F1-Score is the harmonic mean of Precision and Recall, providing a single metric that balances both.
    For NER at the word level, comparing the set of predicted words with their labels against the set of truth words with their labels is a valid approach to assess performance, especially after splitting and unique-ing the data as done previously.
    """
    # Check if either dataframe is empty
    if predicted_df.empty or truth_df.empty:
        return {"Precision": 0, "Recall": 0, "F1-Score": 0}

    # Merge the two dataframes to find matches and mismatches
    merged_df = pd.merge(predicted_df, truth_df, on=['identifier', 'text'], how='outer', indicator=True)

    # True Positives (TP): Present in both predicted and truth
    true_positives = merged_df[merged_df['_merge'] == 'both'].shape[0]

    # False Positives (FP): Present in predicted but not in truth
    false_positives = merged_df[merged_df['_merge'] == 'left_only'].shape[0]

    # False Negatives (FN): Present in truth but not in predicted
    false_negatives = merged_df[merged_df['_merge'] == 'right_only'].shape[0]

    # Calculate Precision, Recall, and F1-Score
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {"Precision": precision, "Recall": recall, "F1-Score": f1_score}

## --- 4. final predictions ---

In [43]:
# Calculate performance for each file using the split and unique dataframes
performance_metrics1 = calculate_ner_performance(pred_text1_split_unique, truth_text1_split_unique)
performance_metrics2 = calculate_ner_performance(pred_text2_split_unique, truth_text2_split_unique)
performance_metrics3 = calculate_ner_performance(pred_text3_split_unique, truth_text3_split_unique)
performance_metrics4 = calculate_ner_performance(pred_text4_split_unique, truth_text4_split_unique)

print("Performance metrics for ARTHROTEC.1 (word level):")
display(performance_metrics1)

print("\nPerformance metrics for ARTHROTEC.2 (word level):")
display(performance_metrics2)

print("\nPerformance metrics for ARTHROTEC.3 (word level):")
display(performance_metrics3)

print("\nPerformance metrics for ARTHROTEC.4 (word level):")
display(performance_metrics4)

Performance metrics for ARTHROTEC.1 (word level):


{'Precision': 1.0, 'Recall': 1.0, 'F1-Score': 1.0}


Performance metrics for ARTHROTEC.2 (word level):


{'Precision': 0.7142857142857143,
 'Recall': 0.9090909090909091,
 'F1-Score': 0.8}


Performance metrics for ARTHROTEC.3 (word level):


{'Precision': 0.8,
 'Recall': 0.3076923076923077,
 'F1-Score': 0.4444444444444444}


Performance metrics for ARTHROTEC.4 (word level):


{'Precision': 0, 'Recall': 0, 'F1-Score': 0}